In [3]:
%run '00_database_connectivity_setup.ipynb'

C:\Anaconda2\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [4]:
%%execsql
DROP TABLE IF EXISTS nate.dectree_output;
DROP TABLE IF EXISTS nate.dectree_output_summary;

The cell below shows the error - if any independent feature column is a boolean, this error appears - complains about syntax near the CASE WHEN statement because True has single quotes around it when it should not perhaps?

In [5]:
%%execsql
SELECT madlib.tree_train('nate.prediction_input', 'nate.dectree_output', 'component', 'pctreplaced',
'component_surface_matl', NULL::text, 'gini', NULL::text, NULL::text, 25, 3, 1, 15)

DatabaseError: Execution failed on sql 'SELECT madlib.tree_train('nate.prediction_input', 'nate.dectree_output', 'component', 'pctreplaced',
'component_surface_matl', NULL::text, 'gini', NULL::text, NULL::text, 25, 3, 1, 15)': plpy.SPIError: syntax error at or near "True" (plpython.c:4648)
LINE 3: ...         '(CASE WHEN component_surface_matl THEN 'True' ELSE...
                                                             ^
QUERY:  
                SELECT
                    '(CASE WHEN component_surface_matl THEN 'True' ELSE 'False' END)' AS colname,
                    levels
                FROM (
                    SELECT
                        '(CASE WHEN component_surface_matl THEN 'True' ELSE 'False' END)' AS colname,
                        array_agg(levels ORDER BY dep_avg) AS levels
                    FROM (
                        SELECT
                            ((CASE WHEN component_surface_matl THEN 'True' ELSE 'False' END))::text AS levels,
                        

Here is a similar statement that runs succesfully, because it does not have any boolean features.

In [6]:
%%execsql
SELECT madlib.tree_train('nate.prediction_input', 'nate.dectree_output', 'component', 'pctreplaced',
'orders_replaced', NULL::text, 'gini', NULL::text, NULL::text, 25, 3, 1, 15)